In [2]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "3"
os.environ["CUDA_VISIBLE_DEVICES"]
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
#!pip3 install pytorch
!pip3 install multiprocess
import torch
torch.cuda.empty_cache()

#!pip3 install sentence-transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator, CEBinaryClassificationEvaluator
from torch.utils.data import DataLoader
import math
import plotly
import plotly.graph_objs as go
import plotly.io as pio
plotly.offline.init_notebook_mode(connected=True)
import multiprocess as multiprocessing
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers import InputExample
import logging
!pip3 install transformers
import json
import pickle
import numpy as np
from tqdm.notebook import trange, tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc
from writeprints import get_writeprints_transformer, prepare_entry
from utills import batch
from pytorch_models import NeuralNet
import torch
import torch.nn as nn
from torch.utils import data
import copy
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from transformers import BertTokenizerFast
import datetime
import time
from pan20_verif_evaluator import evaluate_all
import nltk
nltk.download('punkt')

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package punkt to
[nltk_data]     /storagenfs/g.pisciotta1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Constants
DATA_DIR = 'data/small/'
TEMP_DATA_DIR = 'temp_data/pan20_computed/'

Load ground truth
====

In [6]:
# Load gound truth
ground_truth = {}
partition = {}

#tot = 52601/3
limited = False
tot = 20000
n_of_pos_we_want = tot/2
n_of_neg_we_want = tot/2

# quindi in totale 1_000
positive_samples = 0
negative_samples = 0
total = 0

# Qui apriamo il dataset e estraiamo un totale di 10k esempi del task
with open(DATA_DIR + '/pan20-authorship-verification-training-small-truth.jsonl', 'r') as f:
    
    for counter, l in tqdm(enumerate(f)):
        total += 1
        
        d = json.loads(l)

        if d['same'] and positive_samples < n_of_pos_we_want : # se la label è true e dobbiamo ancora aggiungerne
            ground_truth[d['id']] = d['same']
            positive_samples += 1
        elif not d['same'] and negative_samples < n_of_neg_we_want:
            ground_truth[d['id']] = d['same']
            negative_samples += 1
            
        # Una volta che abbiamo raggiunto il numero di dati che vogliamo, è fatta, usciamo
        if limited and (positive_samples == n_of_pos_we_want and negative_samples == n_of_neg_we_want):
            break
            
print(total, positive_samples, negative_samples)


52601 10000 10000


Partially extract features
===

In [7]:
samples = multiprocessing.Manager().Queue()

from sklearn.model_selection import train_test_split                 
def process_pair(l):
    d = json.loads(l)
    if d['id'] in ground_truth:
        e1 = prepare_entry(d['pair'][0])
        e2 = prepare_entry(d['pair'][1])
        samples.put({'id': d['id'], 'doc1': e1, 'doc2': e2})
        return {'id': d['id'], 'doc1':e1, 'doc2':e2}    


start_time = datetime.datetime.now()
print("Started handling of each pair")
with open(DATA_DIR + 'pan20-authorship-verification-training-small.jsonl', 'r') as f:
    with multiprocessing.Pool() as pool:
        samples = pool.map(process_pair, (l for l in f))
print("Ended handling of each pair")        

end_time = datetime.datetime.now()
print("Time spent: ", (end_time-start_time).total_seconds())



Started handling of each pair
Ended handling of each pair
Time spent:  219.119635


Split dataset
=====

In [8]:
labels = []

# Il processo in parallelo restituisce comunque eventualmente dei None se 
# il sample non appartiene alla lista di quelli che abbiamo selezionato,
# quindi nel caso rimuoviamo i None così
samples = [i for i in samples if i is not None]

for s in samples:
    # Qui non dovrebbe arrivare
    if s is None:
        print("None")
        break
        
    # Questo dovrebbe essere il caso normale
    else:
        s['label'] = int(ground_truth[s['id']])

import pandas as pd
df = pd.DataFrame(samples)
df = df.set_index('id')

X_design, X_test, y_design, y_test = train_test_split(df, df['label'], stratify=df['label'], random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_design, y_design, stratify=y_design, random_state=42)

# Per salvare
X_test.to_pickle(TEMP_DATA_DIR+"x_test.pkl")
X_valid.to_pickle(TEMP_DATA_DIR+"x_valid.pkl")
X_train.to_pickle(TEMP_DATA_DIR+"x_train.pkl")
df.to_pickle(TEMP_DATA_DIR+"df.pkl")


In [9]:
df

,doc1,doc2,label
id,,,
6cced668-6e51-5212-873c-717f2bc91ce6,"{'preprocessed': 'I shift a bit, warily lettin...","{'preprocessed': '""All will become one with Ru...",1
3c6c188a-db28-59aa-8c09-3d0f799ff579,"{'preprocessed': 'I shift a bit, warily lettin...","{'preprocessed': 'Suddenly, a piece of ice fal...",1
b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,{'preprocessed': 'A single tear escaped me as ...,"{'preprocessed': 'got the Yang yoyo."" Kimiko p...",1
e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,"{'preprocessed': '""Ja."" Ludwig kept his gaze u...",{'preprocessed': 'SilverGray lll...YellowRagge...,1
4fe541af-912e-5a86-81a5-94c6d3891509,"{'preprocessed': 'And he did. Slowly, hesitant...","{'preprocessed': '""Let""s go,"" Raimondo said an...",1
...,...,...,...
d244814f-fbf1-5db4-a40c-b973b08ac9f8,{'preprocessed': 'He had a nice voice. Althoug...,{'preprocessed': 'trusty goose-down pillow. Ra...,0
a04024b3-2fba-5ee3-ab06-dac5c10301b6,{'preprocessed': 'I can be with you until 9 p....,"{'preprocessed': '""She likes him more than you...",0
4c5e5e32-79fe-5239-ae1e-a295ca20f6c0,{'preprocessed': 'To help us most to grow If w...,"{'preprocessed': 'Onward to the next chapter, ...",0


Embedding Model
===


## Load the dataset made of pairs of documents

In [9]:
train_samples = []
valid_samples = []
test_samples = []

for i, row_dict in X_train.iterrows():
    train_samples.append(InputExample(texts=[row_dict['doc1']['preprocessed'], row_dict['doc2']['preprocessed']], label=int(ground_truth[i])))
    #train_samples.append(InputExample(texts=[row_dict['doc2']['preprocessed'], row_dict['doc1']['preprocessed']], label=int(ground_truth[i])))

for i, row_dict in X_valid.iterrows():
    valid_samples.append(InputExample(texts=[row_dict['doc1']['preprocessed'], row_dict['doc2']['preprocessed']], label=int(ground_truth[i])))

for i, row_dict in X_test.iterrows():
    test_samples.append(InputExample(texts=[row_dict['doc1']['preprocessed'], row_dict['doc2']['preprocessed']], label=int(ground_truth[i])))

## Test pretrained Cross-Encoder model

In [ ]:
"""model_save_path = 'output/training_stsbenchmark-'+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_name = 'cross-encoder/stsb-TinyBERT-L-4'
model = CrossEncoder(model_name, num_labels=1,device='cuda')

y_pred = []

for row_dict in X_test.to_dict(orient="records"):
    y_pred.append(
        model.predict(
            [row_dict['doc1'], row_dict['doc2']]
        )
    )

print(evaluate_all(y_test, y_pred))"""

## Fine tune pretrained Cross-Encoder model on our dataset

In [10]:
# Load previously trained model
#model_name = 'output/training_stsbenchmark-2021-06-29_16-27-18'

# Fine-tune model
torch.cuda.empty_cache()
train_batch_size = 12
num_epochs = 50
model_save_path = 'output/training_stsbenchmark-'+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# We wrap train_samples (which is a List[InputExample]) into a pytorch DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

# Load preloaded model
model_name = 'cross-encoder/stsb-TinyBERT-L-4'
model = CrossEncoder(model_name, num_labels=1, device='cuda')

# We add an evaluator, which evaluates the performance during training
evaluator = CEBinaryClassificationEvaluator.from_input_examples(valid_samples, name='sts-valid')

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up

model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

##### Load model and eval on test set
#model = CrossEncoder(model_save_path)
evaluator = CEBinaryClassificationEvaluator.from_input_examples(test_samples, name='sts-test')

print("Evaluator: ", evaluator(model))


y_pred = []
for row_dict in X_test.to_dict(orient="records"):
    y_pred.append(
        model.predict(
            [row_dict['doc1']['preprocessed'], row_dict['doc2']['preprocessed']]
        )
    )
print(evaluate_all(y_test, y_pred))




Evaluator:  0.7292112385639142
{'auc': 0.747, 'c@1': 0.696, 'f_05_u': 0.706, 'F1': 0.727, 'overall': 0.719}


In [11]:
from sklearn.metrics import classification_report
new_y_pred = []
for y in y_pred:
    new_y_pred.append(np.round(np.clip(y, 0, 1)))
print(classification_report(y_test, new_y_pred))

              precision    recall  f1-score   support

           0       0.70      0.62      0.66      6192
           1       0.69      0.77      0.73      6959

    accuracy                           0.70     13151
   macro avg       0.70      0.69      0.69     13151
weighted avg       0.70      0.70      0.69     13151

